<a href="https://colab.research.google.com/github/FerBarrey/webscrapingpython/blob/main/Linksvalidations_con_archivos_multiples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install python-docx requests pandas

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
import random

def generar_enlace_aleatorio():
    """Genera un enlace aleatorio, a veces válido, a veces inválido."""
    valido = random.choice([True, False])
    if valido:
        dominio = random.choice(["www.google.com", "www.wikipedia.org", "www.ejemplo.com"])
        ruta = "/pagina" + str(random.randint(1, 10))
        return "https://" + dominio + ruta
    else:
        # Genera un enlace inválido con mayor frecuencia
        dominio = "www.enlacenoexiste" + str(random.randint(1, 1000)) + ".com"
        return "https://" + dominio + "/ruta_invalida"

def generar_lista_enlaces(cantidad):
    """Genera una lista de enlaces aleatorios."""
    return [generar_enlace_aleatorio() for _ in range(cantidad)]

# Generar 20 enlaces aleatorios
enlaces = generar_lista_enlaces(20)

# Crear un DataFrame de pandas con los enlaces
df = pd.DataFrame({"enlaces": enlaces})

# Guardar el DataFrame en un archivo Excel
df.to_excel("links.xlsx", index=False)

print("Archivo links.xlsx generado con enlaces aleatorios.")

Archivo links.xlsx generado con enlaces aleatorios.


In [5]:
# Analisis de los links contenidos en el archivo links.xlsx

import pandas as pd
import requests

def verificar_enlace(enlace):
  """Verifica si un enlace es válido."""
  try:
    response = requests.get(enlace, timeout=5)  # Timeout para evitar esperas largas
    response.raise_for_status()  # Lanza una excepción para códigos de estado no exitosos (4xx o 5xx)
    return "Válido"
  except requests.exceptions.RequestException:
    return "Inválido"

# Cargar el archivo Excel
df = pd.read_excel("links.xlsx")

# Verificar cada enlace y agregar la columna "Válido"
df["Estado"] = df["enlaces"].apply(verificar_enlace)

# Guardar los resultados en un nuevo archivo Excel
df.to_excel("links_verificados.xlsx", index=False)

print("Archivo links_verificados.xlsx generado con los enlaces verificados.")


Archivo links_verificados.xlsx generado con los enlaces verificados.


In [6]:
# Inicio del proceso completo

import docx
!pip install python-docx
import pandas as pd
import requests
import logging

# Configura el registro de errores
logging.basicConfig(filename='error_log.txt', level=logging.ERROR,
                    format='%(asctime)s - %(levelname)s - %(message)s')

def verificar_enlace(url):
    try:
        response = requests.head(url, timeout=5)
        if response.status_code == 200:
            return 'Válido'
        else:
            return 'No Válido'
    except requests.exceptions.RequestException as e:
        logging.error(f"Error al verificar URL {url}: {e}")
        return 'Error'

try:
    doc = docx.Document('Informe.docx')
    links_data = []

    for paragraph in doc.paragraphs:
        for run in paragraph.runs:
            if run.hyperlink:
                text = run.text
                url = run.hyperlink.url
                categoria = 'Otro'
                if 'facebook.com' in url:
                    categoria = 'Facebook'
                elif 'twitter.com' in url or 'x.com' in url:
                    categoria = 'X'
                validez = verificar_enlace(url)
                links_data.append([text, url, categoria, validez])

    df = pd.DataFrame(links_data, columns=['Texto del Link', 'URL', 'Categoría', 'Validez'])
    df.to_excel('linkinformeok.xlsx', index=False)
    print("Archivo 'linkinformeok.xlsx' creado exitosamente.")

except FileNotFoundError:
    logging.error("Archivo 'Informe.docx' no encontrado.")
    print("Error: Archivo 'Informe.docx' no encontrado.")
except Exception as e:
    logging.error(f"Error inesperado: {e}")
    print(f"Error inesperado: {e}")


ERROR:root:Error inesperado: Package not found at 'Informe.docx'


Error inesperado: Package not found at 'Informe.docx'


In [7]:
#Pruebo si se carga correctamente el archivo
import os
print(os.path.exists("Informe1.docx") ) # Debe imprimir True si el archivo está presente


True


In [8]:
from docx import Document

try:
    doc = Document("Informe1.docx")
    print("El archivo se cargó correctamente.")
except Exception as e:
    print(f"Error al abrir el documento: {e}")


El archivo se cargó correctamente.


In [9]:
from docx import Document
from lxml import etree

def get_hyperlinks(doc_path):
    doc = Document(doc_path)
    hyperlinks = []

    for rel in doc.part.rels:
        if "hyperlink" in doc.part.rels[rel].reltype:
            hyperlinks.append(doc.part.rels[rel].target_ref)

    return hyperlinks

doc_path = "Informe1.docx"
links = get_hyperlinks(doc_path)

print("Hipervínculos extraídos:", links)


Hipervínculos extraídos: ['https://www.c5n.com/sociedad/el-drama-los-vecinos-afectados-las-inundaciones-zona-sur-es-toda-la-vida-esto-n192775?fbclid=IwY2xjawIzxs1leHRuA2FlbQIxMQABHXdMs5R4tUGMo8xCHuepjMmCC-6Bu-cvXdimWayLi3cHomKTuj1_ugSe7A_aem_mqgEEd32hdKJpURcnY8E4g', 'https://ultimanoticiaba.com/2025/03/04/demoras-y-anegamientos-tras-intensas-lluvias-en-el-amba/', 'https://www.facebook.com/photo/?fbid=1860890784736076&set=a.118268865664952&locale=es_LA', 'https://www.facebook.com/photo?fbid=1093970502743887&set=pcb.1093970952743842', 'https://www.facebook.com/photo/?fbid=1207190034743817&set=pcb.1207190064743814&locale=es_LA', 'https://junin24.com/locales/mas-de-100-mm-de-lluvia-caida-en-menos-de-15-horas-el-saldo-de-calles-anegadas-bocas-de-tormenta-tapadas-y-el-enojo-de-los-vecinos/?fbclid=IwY2xjawIzyKpleHRuA2FlbQIxMQABHXE-26L6LmPuHWe8JUgxpU08ZYi6-0fvdZia9U6VP7eGFAiq_aN_q1ajvQ_aem_FIG7nuXJgsqYPWQ-KdOuow', 'https://datamagdalena.com.ar/nota/1916/-mas-de-136-mm-de-lluvia-en-bavio-bomber

In [10]:
import pandas as pd
import requests
from docx import Document
from concurrent.futures import ThreadPoolExecutor

def verificar_enlace(url):
    """Verifica si una URL es accesible con un timeout corto."""
    try:
        response = requests.head(url, allow_redirects=True, timeout=1.5)  # Usa HEAD para mayor velocidad
        return 'Válido' if response.status_code == 200 else 'No Válido'
    except requests.exceptions.RequestException:
        return 'Error'

def get_hyperlinks(doc_path):
    """Extrae hipervínculos de un documento Word (.docx)."""
    doc = Document(doc_path)
    return [doc.part.rels[rel].target_ref for rel in doc.part.rels if "hyperlink" in doc.part.rels[rel].reltype]

def procesar_links(doc_path):
    """Extrae hipervínculos, los categoriza y verifica su validez en paralelo."""
    hyperlinks = get_hyperlinks(doc_path)
    links_data = []

    # Clasificar los enlaces rápidamente
    categorias = ['Facebook' if 'facebook.com' in url else
                  'X' if 'twitter.com' in url or 'x.com' in url else 'Otro'
                  for url in hyperlinks]

    # Verificar los enlaces en paralelo (para mayor velocidad)
    with ThreadPoolExecutor(max_workers=10) as executor:
        validez_list = list(executor.map(verificar_enlace, hyperlinks))

    # Combinar los datos en una lista final
    links_data = list(zip(hyperlinks, categorias, validez_list))
    return links_data

# Ejecutar la extracción y guardar en Excel
try:
    doc_path = 'Informe1.docx'
    links_data = procesar_links(doc_path)

    if links_data:
        df = pd.DataFrame(links_data, columns=['URL', 'Categoría', 'Validez'])
        df.to_excel('linkinformeok.xlsx', index=False)
        print("✅ Archivo 'linkinformeok.xlsx' creado exitosamente.")
    else:
        print("⚠️ No se encontraron hipervínculos en el documento.")

except FileNotFoundError:
    print("❌ Error: Archivo 'Informe1.docx' no encontrado.")
except Exception as e:
    print(f"❌ Error inesperado: {e}")


✅ Archivo 'linkinformeok.xlsx' creado exitosamente.


In [11]:
!pip install sumy
!pip install python-docx
!pip install nltk
import pandas as pd
import requests
from docx import Document
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer


#Descargo recursos NLTK
import nltk
nltk.download('punkt')

# Download the 'punkt_tab' data package( esto me asegura que lea en español)
nltk.download('punkt_tab')


def verificar_enlace(url):
    """Verifica si una URL es accesible con un timeout corto."""
    try:
        response = requests.head(url, allow_redirects=True, timeout=1.5)
        return 'Válido' if response.status_code == 200 else 'No Válido'
    except requests.exceptions.RequestException:
        return 'Error'

def get_hyperlinks(doc_path):
    """Extrae hipervínculos de un documento Word (.docx)."""
    doc = Document(doc_path)
    return [doc.part.rels[rel].target_ref for rel in doc.part.rels if "hyperlink" in doc.part.rels[rel].reltype]

def extraer_texto_y_resumir(url, num_sentences=3):
    """Extrae el texto de la noticia desde la URL y genera un resumen relevante."""
    try:
        response = requests.get(url, timeout=3)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extraer el texto principal de la noticia
        paragraphs = soup.find_all('p')
        full_text = ' '.join([p.get_text() for p in paragraphs if len(p.get_text()) > 20])

        if not full_text:
            return "No se pudo extraer el contenido."

        # Resumir el texto usando Sumy LSA (asegurando compatibilidad con el idioma)
        parser = PlaintextParser.from_string(full_text, Tokenizer("es"))  # Alternativa: "spanish"
        summarizer = LsaSummarizer()
        summary = summarizer(parser.document, num_sentences)

        return ' '.join([str(sentence) for sentence in summary])

    except requests.exceptions.RequestException:
        return "No se pudo acceder a la noticia."

def procesar_links(doc_path):
    """Extrae hipervínculos, los categoriza, verifica su validez y resume noticias."""
    hyperlinks = get_hyperlinks(doc_path)

    # Clasificar los enlaces rápidamente
    categorias = ['Facebook' if 'facebook.com' in url else
                  'X' if 'twitter.com' in url or 'x.com' in url else 'Otro'
                  for url in hyperlinks]

    # Verificar los enlaces en paralelo (para mayor velocidad)
    with ThreadPoolExecutor(max_workers=10) as executor:
        validez_list = list(executor.map(verificar_enlace, hyperlinks))

    # Extraer el resumen de cada noticia en paralelo
    with ThreadPoolExecutor(max_workers=5) as executor:
        resumen_list = list(executor.map(extraer_texto_y_resumir, hyperlinks))

    # Combinar los datos en una lista final
    links_data = list(zip(hyperlinks, categorias, validez_list, resumen_list))
    return links_data

# Ejecutar la extracción y guardar en Excel
try:
    doc_path = 'Informe1.docx'
    links_data = procesar_links(doc_path)

    if links_data:
        df = pd.DataFrame(links_data, columns=['URL', 'Categoría', 'Validez', 'Resumen'])
        df.to_excel('linkinformeok.xlsx', index=False)
        print("✅ Archivo 'linkinformeok.xlsx' creado exitosamente con resúmenes.")
    else:
        print("⚠️ No se encontraron hipervínculos en el documento.")

except FileNotFoundError:
    print("❌ Error: Archivo 'Informe.docx' no encontrado.")
except Exception as e:
    print(f"❌ Error inesperado: {e}")


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.5 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21693 sha256=fe3acd499a3c1ab5518c7f240b7d314797a1b1e2223893255a09c38c19cfc55f
  Stored in directory: /root/.cache/pip/wheels/4d/57/58/7e3d7fedf51fe248b7fcee3df6945ae28638e22cddf01eb92b
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=3890bf4bf8c0322b92fb546080c8cbdf83281a76c4a35b0dad62a050641cca7a
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built breadability docopt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


✅ Archivo 'linkinformeok.xlsx' creado exitosamente con resúmenes.


In [12]:
#VERIFICO SI EL ARCHIVO se puede abrir correctamente
from docx import Document
!pip install python-docx

try:
    ruta_archivo = 'Informe1.docx'
    doc = Document(ruta_archivo)  # Abrir el archivo
    print("Archivo abierto correctamente.")
except Exception as e:
    print(f"Error inesperado: {e}")

Archivo abierto correctamente.


In [13]:
#Verifico si el idioma es compatible con el tokenizador
from nltk.tokenize import word_tokenize

texto = "Hola, ¿cómo estás?"
tokens = word_tokenize(texto, language='spanish')
print(tokens)

['Hola', ',', '¿cómo', 'estás', '?']


In [14]:
#verifico si el tokenizador esta instalado correctamente

import nltk
print(nltk.__version__)

"""el tokenizador cumple un papel fundamental en el proceso de resumen de texto que utiliza la biblioteca sumy. Específicamente, el Tokenizer("spanish") se utiliza dentro de la función extraer_texto_y_resumir.Esto es lo que hace:Divide el texto: El tokenizador toma el full_text extraído de una página web y lo divide en unidades individuales, que suelen ser palabras y signos de puntuación. Procesamiento específico del idioma: Al especificar "spanish", el tokenizador se configura para entender las reglas y estructuras del idioma español. Esto es importante para identificar correctamente los límites de las palabras, manejar la puntuación y, potencialmente, reconocer frases o contracciones comunes en español."""
#"""Prepara para el resumen: La salida del tokenizador (una lista de tokens) es utilizada posteriormente por el LsaSummarizer. Los algoritmos de resumen como LSA funcionan analizando las relaciones entre palabras y oraciones. Una tokenización adecuada es esencial para que estos algoritmos procesen el texto con precisión e identifiquen las oraciones más importantes para el resumen.
#En resumen, el tokenizador prepara el texto para el algoritmo de resumen convirtiéndolo a un formato que el algoritmo pueda entender y procesar eficazmente, teniendo en cuenta el idioma específico del texto

3.9.1


'el tokenizador cumple un papel fundamental en el proceso de resumen de texto que utiliza la biblioteca sumy. Específicamente, el Tokenizer("spanish") se utiliza dentro de la función extraer_texto_y_resumir.Esto es lo que hace:Divide el texto: El tokenizador toma el full_text extraído de una página web y lo divide en unidades individuales, que suelen ser palabras y signos de puntuación. Procesamiento específico del idioma: Al especificar "spanish", el tokenizador se configura para entender las reglas y estructuras del idioma español. Esto es importante para identificar correctamente los límites de las palabras, manejar la puntuación y, potencialmente, reconocer frases o contracciones comunes en español.'

# Modifico el codigo anterior para tratamiento multiple de archivos de entrada(docx) y salida( excel)
Modifica el código  para iterar sobre varios documentos de Word (p. ej., "Informe1.docx", "Informe2.docx", "Informe3.docx") y procesa cada uno individualmente con la función `procesar_links`. Para cada documento de entrada, guarda el resultado en un archivo de Excel con un nombre único (p. ej., "linkinformeok1.xlsx", "linkinformeok2.xlsx", "linkinformeok3.xlsx"). .


## Modificar la función `procesar links`

### Subtarea:
Actualizar la función `procesar_links` para que acepte una lista de rutas de documentos como entrada en lugar de una sola ruta.

In [15]:
def procesar_links(doc_paths):
    """Extrae hipervínculos from multiple Word documents, categorizes, verifies validity, and summarizes news."""
    all_links_dataframes = []

    for doc_path in doc_paths:
        try:
            hyperlinks = get_hyperlinks(doc_path)
            links_data = []

            # Clasificar los enlaces rápidamente
            categorias = ['Facebook' if 'facebook.com' in url else
                          'X' if 'twitter.com' in url or 'x.com' in url else 'Otro'
                          for url in hyperlinks]

            # Verificar los enlaces en paralelo (para mayor velocidad)
            with ThreadPoolExecutor(max_workers=10) as executor:
                validez_list = list(executor.map(verificar_enlace, hyperlinks))

            # Extraer el resumen de cada noticia en paralelo
            with ThreadPoolExecutor(max_workers=5) as executor:
                resumen_list = list(executor.map(extraer_texto_y_resumir, hyperlinks))

            # Combinar los datos en una lista final
            links_data = list(zip(hyperlinks, categorias, validez_list, resumen_list))

            if links_data:
                df = pd.DataFrame(links_data, columns=['URL', 'Categoría', 'Validez', 'Resumen'])
                all_links_dataframes.append(df)
            else:
                print(f"⚠️ No se encontraron hipervínculos en el documento: {doc_path}")

        except FileNotFoundError:
            print(f"❌ Error: Archivo '{doc_path}' no encontrado.")
        except Exception as e:
            print(f"❌ Error inesperado al procesar {doc_path}: {e}")

    return all_links_dataframes

## Iterar documentos

### Subtarea:
Modifica el bloque de ejecución principal para obtener una lista de todos los documentos de Word que se procesarán.


In [16]:
document_list = ['Informe1.docx', 'Informe2.docx', 'Informe3.docx', 'Informe4.docx','Informe5.docx']

## Procesa cada documento alojados en documet_list

### Subtarea:
Recorre la lista de documentos, llama a la función procesar_links modificada para cada documento y guarda los resultados en un archivo de Excel con un nombre único ( linkinformeok1.xlsx, linkinformeok2.xlsx). (e.g., `linkinformeok1.xlsx`, `linkinformeok2.xlsx`).


In [17]:
document_list = ['Informe1.docx', 'Informe2.docx', 'Informe3.docx','Informe4.docx','Informe5.docx' ]

# Se ejecuta el proceso sobre el listado de documentos
all_processed_dataframes = procesar_links(document_list)

# Se guarda cada nuevo archivo excel con un nombre y su numero correspondiente
for i, df in enumerate(all_processed_dataframes):
    output_filename = f'linkinformeok{i+1}.xlsx'
    try:
        df.to_excel(output_filename, index=False)
        print(f"✅ Archivo '{output_filename}' creado exitosamente.")
    except Exception as e:
        print(f"❌ Error al guardar el archivo '{output_filename}': {e}")

if not all_processed_dataframes:
    print("⚠️ No se procesaron documentos o no se encontraron hipervínculos válidos en ninguno.")


✅ Archivo 'linkinformeok1.xlsx' creado exitosamente.
✅ Archivo 'linkinformeok2.xlsx' creado exitosamente.
✅ Archivo 'linkinformeok3.xlsx' creado exitosamente.
✅ Archivo 'linkinformeok4.xlsx' creado exitosamente.
✅ Archivo 'linkinformeok5.xlsx' creado exitosamente.


In [18]:
import pandas as pd
import requests
from docx import Document
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
import nltk

# Download necessary NLTK data (ensure they are downloaded)
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')
try:
    nltk.data.find('tokenizers/punkt_tab')
except nltk.downloader.DownloadError:
    nltk.download('punkt_tab')


def verificar_enlace(url):
    """Verifica si una URL es accesible con un timeout corto."""
    try:
        response = requests.head(url, allow_redirects=True, timeout=1.5)  # Usa HEAD para mayor speed
        return 'Válido' if response.status_code == 200 else 'No Válido'
    except requests.exceptions.RequestException:
        return 'Error'

def get_hyperlinks(doc_path):
    """Extrae hipervínculos de un documento Word (.docx)."""
    doc = Document(doc_path)
    return [doc.part.rels[rel].target_ref for rel in doc.part.rels if "hyperlink" in doc.part.rels[rel].reltype]

def extraer_texto_y_resumir(url, num_sentences=3):
    """Extrae el texto de la noticia desde la URL and genera un resumen relevante."""
    try:
        response = requests.get(url, timeout=3)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extraer el texto principal de la noticia (adaptable según la web)
        paragraphs = soup.find_all('p')
        full_text = ' '.join([p.get_text() for p in paragraphs if len(p.get_text()) > 20])

        if not full_text:
            return "No se pudo extraer el contenido."

        # Resumir el texto usando Sumy LSA
        parser = PlaintextParser.from_string(full_text, Tokenizer("spanish"))
        summarizer = LsaSummarizer()
        summary = summarizer(parser.document, num_sentences)

        return ' '.join([str(sentence) for sentence in summary])

    except requests.exceptions.RequestException:
        return "No se pudo acceder a la noticia."

def procesar_links(doc_paths):
    """Extrae hipervínculos from multiple Word documents, categorizes, verifies validity, and summarizes news."""
    all_links_dataframes = []

    for doc_path in doc_paths:
        try:
            hyperlinks = get_hyperlinks(doc_path)
            links_data = []

            # Clasificar los enlaces rápidamente
            categorias = ['Facebook' if 'facebook.com' in url else
                          'X' if 'twitter.com' in url or 'x.com' in url else 'Otro'
                          for url in hyperlinks]

            # Verificar los enlaces en paralelo (para mayor velocidad)
            with ThreadPoolExecutor(max_workers=10) as executor:
                validez_list = list(executor.map(verificar_enlace, hyperlinks))

            # Extraer el resumen de cada noticia en paralelo
            with ThreadPoolExecutor(max_workers=5) as executor:
                resumen_list = list(executor.map(extraer_texto_y_resumir, hyperlinks))

            # Combinar los datos en una lista final
            links_data = list(zip(hyperlinks, categorias, validez_list, resumen_list))

            if links_data:
                df = pd.DataFrame(links_data, columns=['URL', 'Categoría', 'Validez', 'Resumen'])
                all_links_dataframes.append(df)
            else:
                print(f"⚠️ No se encontraron hipervínculos en el documento: {doc_path}")

        except FileNotFoundError:
            print(f"❌ Error: Archivo '{doc_path}' no encontrado.")
        except Exception as e:
            print(f"❌ Error inesperado al procesar {doc_path}: {e}")

    return all_links_dataframes

# List of documents to process
document_list = ['Informe1.docx', 'Informe2.docx', 'Informe3.docx', 'Informe4.docx','Informe5.docx']

# Execute the processing for the list of documents
all_processed_dataframes = procesar_links(document_list)

# Save each dataframe to a uniquely named Excel file
for i, df in enumerate(all_processed_dataframes):
    output_filename = f'linkinformeok{i+1}.xlsx'
    try:
        df.to_excel(output_filename, index=False)
        print(f"✅ Archivo '{output_filename}' creado exitosamente.")
    except Exception as e:
        print(f"❌ Error al guardar el archivo '{output_filename}': {e}")

if not all_processed_dataframes:
    print("⚠️ No se procesaron documentos o no se encontraron hipervínculos válidos en ninguno.")


✅ Archivo 'linkinformeok1.xlsx' creado exitosamente.
✅ Archivo 'linkinformeok2.xlsx' creado exitosamente.
✅ Archivo 'linkinformeok3.xlsx' creado exitosamente.
✅ Archivo 'linkinformeok4.xlsx' creado exitosamente.
✅ Archivo 'linkinformeok5.xlsx' creado exitosamente.


In [19]:
!pip install python-docx

In [20]:
import pandas as pd
import requests
from docx import Document
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
import nltk

# Download necessary NLTK data (ensure they are downloaded)
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')
try:
    nltk.data.find('tokenizers/punkt_tab')
except nltk.downloader.DownloadError:
    nltk.download('punkt_tab')

def verificar_enlace(url):
    """Verifica si una URL es accesible con un timeout corto."""
    try:
        response = requests.head(url, allow_redirects=True, timeout=1.5)  # Usa HEAD para mayor speed
        return 'Válido' if response.status_code == 200 else 'No Válido'
    except requests.exceptions.RequestException:
        return 'Error'

def get_hyperlinks(doc_path):
    """Extrae hipervínculos de un documento Word (.docx)."""
    doc = Document(doc_path)
    return [doc.part.rels[rel].target_ref for rel in doc.part.rels if "hyperlink" in doc.part.rels[rel].reltype]

def extraer_texto_y_resumir(url, num_sentences=3):
    """Extrae el texto de la noticia desde la URL and genera un resumen relevante."""
    try:
        response = requests.get(url, timeout=3)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extraer el texto principal de la noticia (adaptable según la web)
        paragraphs = soup.find_all('p')
        full_text = ' '.join([p.get_text() for p in paragraphs if len(p.get_text()) > 20])

        if not full_text:
            return "No se pudo extraer el contenido."

        # Resumir el texto usando Sumy LSA
        parser = PlaintextParser.from_string(full_text, Tokenizer("spanish"))
        summarizer = LsaSummarizer()
        summary = summarizer(parser.document, num_sentences)

        return ' '.join([str(sentence) for sentence in summary])

    except requests.exceptions.RequestException:
        return "No se pudo acceder a la noticia."

def procesar_links(doc_paths):
    """Extrae hipervínculos from multiple Word documents, categorizes, verifies validity, and summarizes news."""
    all_links_dataframes = []

    for doc_path in doc_paths:
        try:
            hyperlinks = get_hyperlinks(doc_path)
            links_data = []

            # Clasificar los enlaces rápidamente
            categorias = ['Facebook' if 'facebook.com' in url else
                          'X' if 'twitter.com' in url or 'x.com' in url else 'Otro'
                          for url in hyperlinks]

            # Verificar los enlaces en paralelo (para mayor velocidad)
            with ThreadPoolExecutor(max_workers=10) as executor:
                validez_list = list(executor.map(verificar_enlace, hyperlinks))

            # Extraer el resumen de cada noticia en paralelo
            with ThreadPoolExecutor(max_workers=5) as executor:
                resumen_list = list(executor.map(extraer_texto_y_resumir, hyperlinks))

            # Combinar los datos en una lista final
            links_data = list(zip(hyperlinks, categorias, validez_list, resumen_list))

            if links_data:
                df = pd.DataFrame(links_data, columns=['URL', 'Categoría', 'Validez', 'Resumen'])
                all_links_dataframes.append(df)
            else:
                print(f"⚠️ No se encontraron hipervínculos en el documento: {doc_path}")

        except FileNotFoundError:
            print(f"❌ Error: Archivo '{doc_path}' no encontrado.")
        except Exception as e:
            print(f"❌ Error inesperado al procesar {doc_path}: {e}")

    return all_links_dataframes

# List of documents to process
document_list = ['Informe1.docx', 'Informe2.docx', 'Informe3.docx', 'Informe4.docx','Informe5.docx']

# Execute the processing for the list of documents
all_processed_dataframes = procesar_links(document_list)

# Save each dataframe to a uniquely named Excel file
for i, df in enumerate(all_processed_dataframes):
    output_filename = f'linkinformeok{i+1}.xlsx'
    try:
        df.to_excel(output_filename, index=False)
        print(f"✅ Archivo '{output_filename}' creado exitosamente.")
    except Exception as e:
        print(f"❌ Error al guardar el archivo '{output_filename}': {e}")

if not all_processed_dataframes:
    print("⚠️ No se procesaron documentos o no se encontraron hipervínculos válidos en ninguno.")

✅ Archivo 'linkinformeok1.xlsx' creado exitosamente.
✅ Archivo 'linkinformeok2.xlsx' creado exitosamente.
✅ Archivo 'linkinformeok3.xlsx' creado exitosamente.
✅ Archivo 'linkinformeok4.xlsx' creado exitosamente.
✅ Archivo 'linkinformeok5.xlsx' creado exitosamente.


In [21]:
!pip install sumy #con esta libreria se resumen los textos.